In [1]:
import os

pdf_directory = os.path.join(os.getcwd(), "셀트리온 공시자료")

pdf_files = [os.path.splitext(f)[0] for f in os.listdir(pdf_directory) if f.lower().endswith('.pdf')]

print(f"총 {len(pdf_files)}개의 PDF 파일을 찾았습니다.")
print("\n파일 목록:")
for i, file in enumerate(pdf_files, 1):
    print(f"{i}. {file}")

print("\n파일 리스트 변수:")
print(pdf_files)

총 84개의 PDF 파일을 찾았습니다.

파일 목록:
1. [셀트리온][정정]주요사항보고서(무상증자결정)(2025.05.28)
2. [셀트리온][정정]주요사항보고서(자기주식취득결정)(2025.05.28)
3. [셀트리온]대규모기업집단현황공시[분기별공시(개별회사용)](2024.12.02)
4. [셀트리온]대규모기업집단현황공시[분기별공시(개별회사용)](2025.02.28)
5. [셀트리온]대규모기업집단현황공시[연1회공시및1_4분기용(개별회사)](2025.05.30)
6. [셀트리온]분기보고서(2025.05.15)-1
7. [셀트리온]분기보고서(2025.05.15)-10
8. [셀트리온]분기보고서(2025.05.15)-11
9. [셀트리온]분기보고서(2025.05.15)-12
10. [셀트리온]분기보고서(2025.05.15)-13
11. [셀트리온]분기보고서(2025.05.15)-14
12. [셀트리온]분기보고서(2025.05.15)-15
13. [셀트리온]분기보고서(2025.05.15)-16
14. [셀트리온]분기보고서(2025.05.15)-17
15. [셀트리온]분기보고서(2025.05.15)-18
16. [셀트리온]분기보고서(2025.05.15)-19
17. [셀트리온]분기보고서(2025.05.15)-2
18. [셀트리온]분기보고서(2025.05.15)-20
19. [셀트리온]분기보고서(2025.05.15)-21
20. [셀트리온]분기보고서(2025.05.15)-22
21. [셀트리온]분기보고서(2025.05.15)-23
22. [셀트리온]분기보고서(2025.05.15)-24
23. [셀트리온]분기보고서(2025.05.15)-25
24. [셀트리온]분기보고서(2025.05.15)-3
25. [셀트리온]분기보고서(2025.05.15)-4
26. [셀트리온]분기보고서(2025.05.15)-5
27. [셀트리온]분기보고서(2025.05.15)-6
28. [셀트리온]분기보고서(2025.05.15)-7
29. [셀트리온]분기보고서(2025.05.15)-8
30. [셀트

In [ ]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModel
import requests
import time
import urllib.parse
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# --- 1. OCR 텍스트 추출 함수 (API 요청 방식으로 수정) ---
def extract_text_from_clova_ocr(file_name):
    """
    Naver Clova OCR API를 호출하여, 응답에서 텍스트를 추출하고 재구성합니다.
    """

    base_url = "https://kr.object.ncloudstorage.com/scd.kjsoo/"
    extension = ".pdf"
    encoded_file_name = urllib.parse.quote(file_name)
    
    # API 요청 정보 - 환경 변수에서 가져오기
    api_url = os.getenv("CLOVA_OCR_API_URL")
    headers = {
        "Content-Type": "application/json",
        "X-OCR-SECRET": os.getenv("CLOVA_OCR_SECRET")
    }
    
    # 요청 본문 구성
    body = {
        "version": "V2",
        "requestId": "1234",
        "timestamp": str(int(time.time() * 1000)),  # 현재 시간(밀리초)
        "lang": "ko",
        "images": [
            {
                "format": "pdf",
                "name": file_name,
                "url": base_url + encoded_file_name + extension,
            }
        ],
        "enableTableDetection": False
    }
    
    print(body)

    try:
        # API 요청 보내기
        response = requests.post(api_url, headers=headers, json=body)
        response.raise_for_status()  # 오류 상태 코드에 대한 예외 발생
        
        # 응답 파싱
        ocr_data = response.json()
    except requests.RequestException as e:
        print(f"API 요청 오류: {e}")
        return None
    except json.JSONDecodeError:
        print(f"응답 JSON 파싱 실패")
        return None
    
    full_text = ""
    images = ocr_data.get("images", [])
    
    if not images:
        print("오류: JSON 데이터에서 'images' 필드를 찾을 수 없습니다.")
        return ""

    for image in images:
        fields = image.get("fields", [])
        if not fields:
            continue

        page_text = ""
        for field in fields:
            text = field.get("inferText", "")
            line_break = field.get("lineBreak", False)

            page_text += text
            if line_break:
                page_text += "\n"
            else:
                page_text += " "

        full_text += page_text.strip() + "\n\n"

    return full_text.strip()

In [ ]:
# 0. Milvus 연결 설정 (환경 변수에서 로드)
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")  # 환경 변수에서 가져오거나 기본값 사용
MILVUS_PORT = os.getenv("MILVUS_PORT", "19530")      # 환경 변수에서 가져오거나 기본값 사용
COLLECTION_NAME = "celltrion_embeddings" # 사용할 컬렉션 이름
VECTOR_DIM = 768           # 임베딩 벡터 차원 (KLUE-BERT base는 768)

# 1. Milvus 연결
print(f"Connecting to Milvus ({MILVUS_HOST}:{MILVUS_PORT})...")
try:
    connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
    print("Successfully connected to Milvus.")
except Exception as e:
    print(f"Failed to connect to Milvus: {e}")
    exit() # 연결 실패 시 종료

# 2. 컬렉션 스키마 정의
# 필드: id (Primary Key), text (원본 텍스트 청크), embedding (벡터)
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    # VARCHAR 최대 길이는 65535, 청크 길이에 따라 조절 가능
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=VECTOR_DIM)
]
schema = CollectionSchema(fields, description="OCR Document Text Chunks and Embeddings")

# 3. 컬렉션 생성 (이미 존재하면 스킵)
if utility.has_collection(COLLECTION_NAME):
    print(f"Collection '{COLLECTION_NAME}' already exists.")
    collection = Collection(COLLECTION_NAME)
else:
    print(f"Creating collection '{COLLECTION_NAME}'...")
    collection = Collection(name=COLLECTION_NAME, schema=schema)
    print(f"Collection '{COLLECTION_NAME}' created successfully.")


# --- 2. 텍스트 추출 및 기본 정제 ---
for file_name in pdf_files:
    reconstructed_text = extract_text_from_clova_ocr(file_name)

    if reconstructed_text is None:
        exit() # 오류 시 종료

    # 기본적인 전처리: 연속된 공백/개행을 하나로 줄이기 (선택적)
    # 이 단계는 청킹 방식에 따라 조절할 수 있습니다. 여기서는 유지합니다.
    cleaned_text = re.sub(r'\s+', ' ', reconstructed_text).strip()
    print("--- 기본 정제된 텍스트 (일부 미리보기) ---")
    print(cleaned_text[:500] + "..." if len(cleaned_text) > 500 else cleaned_text)


    # --- 3. 모델 및 토크나이저 로드 ---
    model_name = "klue/bert-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"\nUsing device: {device}")

    # --- 4. 텍스트 분할 (Fixed-size chunking with overlap) ---
    # 청크 크기와 겹침 크기 설정 (토큰 기준)
    chunk_size = 400  # 모델 최대 길이(512)보다 작게 설정 (special token 등 고려)
    overlap_size = 50   # 청크 간 겹치는 토큰 수

    print(f"\nSplitting text into fixed-size chunks (size={chunk_size}, overlap={overlap_size})...")

    # 전체 텍스트를 한 번에 토큰화 (special token 없이)
    tokens = tokenizer.encode(cleaned_text, add_special_tokens=False)
    total_tokens = len(tokens)
    print(f"Total tokens: {total_tokens}")

    chunks = []
    start_index = 0
    while start_index < total_tokens:
        end_index = start_index + chunk_size
        # 마지막 청크는 end_index가 total_tokens를 넘어도 괜찮음 (슬라이싱이 처리)
        chunk_tokens = tokens[start_index:end_index]

        # 토큰을 다시 텍스트로 디코딩 (special token 없이)
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)

        if chunk_text.strip(): # 내용이 있는 청크만 추가
            chunks.append(chunk_text.strip())

        # 다음 청크 시작 위치 계산 (겹침 고려)
        start_index += (chunk_size - overlap_size)

        # 무한 루프 방지 (만약 step이 0 이하일 경우)
        if chunk_size - overlap_size <= 0:
            print("오류: chunk_size가 overlap_size보다 커야 합니다.")
            break

    print(f"Number of chunks created: {len(chunks)}")
    if chunks:
        print("First chunk preview:", chunks[0][:200] + "..." if len(chunks[0]) > 200 else chunks[0])

    print("\n--- Generated Chunks (Full Preview) ---")
    for i, chunk_text in enumerate(chunks):
        print(f"===== Chunk {i+1}/{len(chunks)} =====")
        # 청크 전체 내용을 보고 싶으면 아래 주석 해제 (너무 길면 일부만 출력)
        print(chunk_text)


    # --- 5. Mean Pooling 함수 정의 --- (이전과 동일)
    def mean_pooling(model_output, attention_mask):
        """Mean Pooling 계산 함수 (Padding 토큰 제외)"""
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    # --- 6. 각 청크를 벡터화 --- (이전과 동일, 입력이 chunks 리스트)
    chunk_vectors = []
    model.eval()

    print("\nStarting vectorization...")
    with torch.no_grad():
        for i, chunk in enumerate(chunks):
            if not chunk:
                print(f"Skipping empty chunk {i+1}/{len(chunks)}")
                continue
            print(f"Processing chunk {i+1}/{len(chunks)}")

            # 토큰화 시에는 모델 최대 길이에 맞춰 truncation/padding 필요
            encoded_input = tokenizer(chunk,
                                    padding=True,
                                    truncation=True,
                                    max_length=512,      # 모델 최대 길이
                                    return_tensors='pt').to(device)

            try:
                model_output = model(**encoded_input)
                sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
                chunk_vectors.append(sentence_embedding.cpu())
            except Exception as e:
                print(f"Error processing chunk {i+1}: {e}")
                print(f"  Chunk content (first 100 chars): {chunk[:100]}")
                chunk_vectors.append(None)

    # --- 7. 결과 확인 --- (이전과 동일)
    valid_vectors = [v for v in chunk_vectors if v is not None]
    print(f"\nSuccessfully vectorized {len(valid_vectors)} chunks (out of {len(chunks)} total).")
    if valid_vectors:
        print("Shape of the vector for the first valid chunk:", valid_vectors[0].shape)
        print("Vector preview for the first valid chunk (first 10 elements):")
        print(valid_vectors[0][0, :10].tolist()) # 첫 번째 벡터의 앞 10개 숫자 출력


    # ===== 섹션 8: Milvus에 데이터 저장 =====
    print("\n--- Section 8: Storing data in Milvus ---")

    # 4. 데이터 준비 (삽입 형식에 맞게)
    # chunk_vectors에는 None이 포함될 수 있으므로, chunks와 함께 순회하며 유효한 데이터만 추출
    entities_to_insert = []
    texts_to_insert = []
    embeddings_to_insert = []

    print("Preparing data for Milvus insertion...")
    num_valid_pairs = 0
    for i, chunk_text in enumerate(chunks):
        if i < len(chunk_vectors) and chunk_vectors[i] is not None:
            # 원본 텍스트
            texts_to_insert.append(chunk_text)
            # 벡터 (PyTorch Tensor -> list)
            vector = chunk_vectors[i].squeeze().tolist() # (1, 768) -> (768,) list
            embeddings_to_insert.append(vector)
            num_valid_pairs += 1
        # else:
        #     print(f"Skipping chunk {i+1} due to missing or invalid vector.")

    if not texts_to_insert:
        print("No valid data to insert into Milvus.")
    else:
        # Milvus 삽입 형식: 각 필드별 리스트를 묶은 리스트
        entities_to_insert = [
            texts_to_insert,
            embeddings_to_insert
        ]
        print(f"Prepared {num_valid_pairs} valid data entities for insertion.")

        # 5. 데이터 삽입
        try:
            print(f"Inserting {len(texts_to_insert)} entities into '{COLLECTION_NAME}'...")
            insert_result = collection.insert(entities_to_insert)
            print(f"Insertion successful. Primary keys: {insert_result.primary_keys[:10]}...") # 처음 10개 PK만 출력

            # (선택적) 데이터 플러시: 디스크에 즉시 쓰도록 강제 (작은 데이터셋에 유용)
            print("Flushing data...")
            collection.flush()
            print("Data flushed.")

        except Exception as e:
            print(f"Failed to insert data into Milvus: {e}")

        # 6. 벡터 필드 인덱스 생성 (이미 존재하면 스킵)
        INDEX_PARAM = {
            "metric_type": "L2",      # 거리 계산 방식 (L2: 유클리드 거리, IP: 내적) - BERT에는 L2 또는 IP
            "index_type": "IVF_FLAT", # 인덱스 종류 (IVF_FLAT, HNSW 등)
            "params": {"nlist": 128}  # IVF_FLAT 파라미터 (nlist 값은 데이터 크기에 따라 조절)
        }

        if not collection.has_index():
            print(f"Creating index ({INDEX_PARAM['index_type']}) for embedding field...")
            try:
                collection.create_index(
                    field_name="embedding",
                    index_params=INDEX_PARAM
                )
                print("Index created successfully.")
            except Exception as e:
                print(f"Failed to create index: {e}")
        else:
            print("Index already exists.")

        # 7. 컬렉션 로드 (검색 준비)
        print("Loading collection into memory...")
        try:
            collection.load()
            print(f"Collection '{COLLECTION_NAME}' loaded successfully.")
            # 로드 상태 확인 (선택적)
            # print(utility.loading_progress(COLLECTION_NAME))
        except Exception as e:
            print(f"Failed to load collection: {e}")

    # 8. Milvus 연결 종료 (선택적)
    # connections.disconnect("default")
    # print("Disconnected from Milvus.")

    print("\n--- Milvus processing finished ---")

Connecting to Milvus (localhost:19530)...
Successfully connected to Milvus.
Collection 'celltrion_embeddings' already exists.
{'version': 'V2', 'requestId': '1234', 'timestamp': '1748800738539', 'lang': 'ko', 'images': [{'format': 'pdf', 'name': '[셀트리온][정정]주요사항보고서(무상증자결정)(2025.05.28)', 'url': 'https://kr.object.ncloudstorage.com/scd.kjsoo/%5B%EC%85%80%ED%8A%B8%EB%A6%AC%EC%98%A8%5D%5B%EC%A0%95%EC%A0%95%5D%EC%A3%BC%EC%9A%94%EC%82%AC%ED%95%AD%EB%B3%B4%EA%B3%A0%EC%84%9C%28%EB%AC%B4%EC%83%81%EC%A6%9D%EC%9E%90%EA%B2%B0%EC%A0%95%29%282025.05.28%29.pdf'}], 'enableTableDetection': False}
API 요청 오류: 400 Client Error: Bad Request for url: https://wgqhbblm1n.apigw.ntruss.com/custom/v1/40105/5047cc3a2269b50314602241ca0d4f616a781c2df6e7b3633e06b68f9eabb019/general


TypeError: expected string or bytes-like object, got 'NoneType'

: 

In [ ]:
from pymilvus import connections, Collection, utility
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# --- Milvus 연결 설정 (환경 변수에서 가져오기) ---
MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")  # 환경 변수에서 가져오거나 기본값 사용
MILVUS_PORT = os.getenv("MILVUS_PORT", "19530")      # 환경 변수에서 가져오거나 기본값 사용
COLLECTION_NAME = "celltrion_embeddings" # 이전에 사용한 컬렉션 이름

# --- 연결 ---
print(f"Connecting to Milvus ({MILVUS_HOST}:{MILVUS_PORT})...")
try:
    connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
    print("Successfully connected to Milvus.")
except Exception as e:
    print(f"Failed to connect to Milvus: {e}")
    exit()

# --- 컬렉션 객체 가져오기 ---
if not utility.has_collection(COLLECTION_NAME):
    print(f"Collection '{COLLECTION_NAME}' does not exist.")
    exit()

collection = Collection(COLLECTION_NAME)
print(f"Got collection object for '{COLLECTION_NAME}'.")

# --- 검증 1: 엔티티 개수 확인 ---
# 중요: flush 이후에도 num_entities가 즉시 업데이트되지 않을 수 있습니다.
#      정확한 개수를 보려면 잠시 기다리거나, 검색/쿼리 전에 collection.load()가 완료되어야 합니다.
try:
    # 데이터 로드를 먼저 수행 (검색/쿼리 전 필수)
    print("Loading collection for verification...")
    collection.load()
    print("Collection loaded.")

    # 로드 후 엔티티 개수 확인
    count_after_load = collection.num_entities
    print(f"Number of entities in collection '{COLLECTION_NAME}': {count_after_load}")
    # 이전에 삽입 시도했던 개수(num_valid_pairs)와 비교해 볼 수 있습니다.
    # print(f"(Expected based on insertion log: {num_valid_pairs})") # num_valid_pairs 변수가 접근 가능하다면

except Exception as e:
    print(f"Error during verification (count/load): {e}")


# --- 검증 2: 샘플 데이터 조회 및 확인 ---
try:
    print("\nRetrieving sample data (first 5 entities)...")
    # limit=5 : 최대 5개의 결과만 가져옴
    # output_fields=["id", "text"] : id와 text 필드만 조회 (embedding은 너무 크므로 제외)
    results = collection.query(
        expr="",  # 모든 데이터를 대상으로 함 (필터링 없음)
        limit=5,
        output_fields=["id", "text"]
    )

    if not results:
        print("No results found in the collection.")
    else:
        print("Sample data:")
        for i, hit in enumerate(results):
            print(f"--- Entity {i+1} ---")
            print(f"  ID: {hit['id']}")
            # text 필드 내용 앞부분만 출력 (너무 길 경우)
            text_preview = hit['text'][:200] + "..." if len(hit['text']) > 200 else hit['text']
            print(f"  Text Preview: {text_preview}")

except Exception as e:
    print(f"Error during querying sample data: {e}")

# --- 연결 종료 ---
# connections.disconnect("default")
# print("\nDisconnected from Milvus.")

Connecting to Milvus (localhost:19530)...
Successfully connected to Milvus.
Got collection object for 'celltrion_embeddings'.
Loading collection for verification...
Collection loaded.
Number of entities in collection 'celltrion_embeddings': 588

Retrieving sample data (first 5 entities)...
Sample data:
--- Entity 1 ---
  ID: 456911237736039155
  Text Preview: 정정신고 ( 보고 ) 정정일자 2024 - 11 - 27 1. 정정관련 공시서류 기업설명회 ( IR ) 개최 ( 안내공시 ) 2. 정정관련 공시서류제출일 2024 - 11 - 26 3. 정정사유 진행방식 변경에 따른 문구 정정 4. 정정사항 정정항목 정정전 정정후 - 해외 IR 행사인 관계로 자료발 - 해외 IR 행사인 관계로 자료발 표는 영어로 진행될 예...
--- Entity 2 ---
  ID: 456911237736039156
  Text Preview: ##시 유튜브 생방송으 로 진행 예정입니다. 생방송 링크는 아래 참고 부탁드립니다. ( https : / / www. youtube. com / live / 8mRJEmbjizcf si = x3FpobSVs3BVTNv ) 9. 기타 투자판단과 관련한 중요사 해외 IR 행사인 관계로 자료발표 및 질의응답 모두 국영문 통역 항 으로 진행될 예정이니 이 점 양해...
--- Entity 3 ---
  ID: 456911237736039158
  Text Preview: 정정신고 ( 보고 ) 정정일자 2024 - 11 - 12 1. 정정관련 공시서류 신규시설투자등 2. 정정관련 공시서류제출일 2020 - 11 - 18 3. 정정사유 공사 정산 완료에 따른 투자금액 정정 4